In [1]:
# missing data (onsite)
# gazette
# wsj

In [130]:
import requests
import sqlite3

from bs4 import BeautifulSoup
from newsplease import NewsPlease

from tqdm import tqdm

from pipeline.db import Database

# BuzzFeed News

In [88]:
con = sqlite3.connect("./data/buzzfeednews.db")
db = Database(con)
parsed = db.read_table("parsed_articles")
onsite = db.get_urls_from_table("onsite")
# check if pub date is none
nopub = parsed[(parsed.pub_date.isnull()) & (parsed.url.isin(onsite))]

In [30]:
pub_dates = []
for k,v in tqdm(nopub.iterrows()):
    u = v.url_canonical
    r = requests.get(u)
    soup = BeautifulSoup(r.text)
    try:
        pub_dt = [i['datetime'] for i in soup.find_all("time") if "Posted" in i.text][0]
    except (AttributeError, IndexError) as e:
        pub_dt = ""
    pub_dates.append(pub_dt)

346it [02:22,  2.42it/s]


In [34]:
nopub.pub_date = pub_dates

c:\Users\Nick\anaconda3\envs\archive\lib\site-packages\pandas\core\generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [39]:
nopub.drop(columns="index").to_sql("parsed_articles", con, if_exists="append")

In [41]:
nopub.sort_values(by="pub_date")

,index,url,url_canonical,text,hed,pub_date
37855,0,https://www.buzzfeednews.com/article/clarissaj...,https://www.buzzfeednews.com/article/clarissaj...,Biden declared “we’re going to win this race” ...,Updates: Joe Biden Has Won The Presidential El...,
14536,0,https://www.buzzfeednews.com/article/buzzfeedn...,https://www.buzzfeednews.com/article/buzzfeedn...,"Live Results: Presidential, Senate, And House ...","Live Results: Presidential, Senate, And House ...",
4014,0,https://www.buzzfeednews.com/article/janelytvy...,https://www.buzzfeednews.com/article/janelytvy...,Election Day is here. We may see a flood of fa...,Here's A Running List Of False And Misleading ...,
37242,0,https://www.buzzfeednews.com/article/tasneemna...,https://www.buzzfeednews.com/article/tasneemna...,BuzzFeed News has journalists around the US br...,"Dozens Of Trump Supporters ""Ambushed"" A Biden ...",
48006,0,https://www.buzzfeednews.com/article/ariannare...,https://www.buzzfeednews.com/article/ariannare...,"Paul Auster, Salman Rushdie, Rebecca Solnit, a...",Here Are Some Great Virtual Book Events Happen...,2020-11-01T16:21:12.000Z
...,...,...,...,...,...,...
44293,0,https://www.buzzfeednews.com/article/laurenstr...,https://www.buzzfeednews.com/article/laurenstr...,"A hospital in Salem, Oregon, said Saturday tha...",Salem Nurse Put On Leave For Anti-Mask COVID T...,2020-11-29T20:36:48.000Z
46001,0,https://www.buzzfeednews.com/article/laurenstr...,https://www.buzzfeednews.com/article/laurenstr...,President-elect Joe Biden has hairline fractur...,Joe Biden Has Hairline Fractures In His Foot A...,2020-11-29T21:49:43.000Z
12700,0,https://www.buzzfeednews.com/article/tasneemna...,https://www.buzzfeednews.com/article/tasneemna...,"The UK's culture secretary, Oliver Dowden, is ...",People Are Amused That The UK Government Wants...,2020-11-29T23:05:18.000Z
38154,0,https://www.buzzfeednews.com/article/azeenghor...,https://www.buzzfeednews.com/article/azeenghor...,The drug company Moderna announced that it wou...,Moderna Has Submitted Its Coronavirus Vaccine ...,2020-11-30T13:32:39.000Z


# Gazette

In [145]:
con = sqlite3.connect("./data/gazette.db")
db = Database(con)
all_urls = db.get_url_superset()

In [146]:
len(all_urls)

438416

In [147]:
all_urls

{'http://gazette.com/81-year-old-italy-native-first-to-buy-legal-pot-in-anchorage/article/feed/432442',
 'https://gazette.com/_services/v1/client_captcha/challenge?request=X2xiX3JhdGVfZm9yZWlnbjpMMmhwWjJndGMyTm9iMjlzTFhOd2IzSjBjeTl3Y21Wd2N5MXliM1Z1WkhWd0wzUjFaWE5rWVhsekxYQnlaWEF0Y205MWJtUjFjQzF3YVd0bGN5MXdaV0ZyTFdOb2NtbHpkR2xoYmkxaVpYTjBjeTF0YVdGdGFTMTViMlJsY2kxMGJ5MW5jbUZpTFdacGNuTjBMWEp2ZFc1a0xYWnBZM1J2Y25rdllYSjBhV05zWlY4MllqZzFaakEyWVMwNU0yWXdMVEV4WldNdFlUYzROaTAyWWpSbE1HWTBObVptWmpRdVlXMXdMbWgwYld3OjE2NDU2MzMyMDE6MHg5OTk1ODlmMjdiYzljNmVhMmM4ZjJhNTg4MDhkZDhiM2RjOTk0NDY3',
 'http://gazette.com:80/cbi-investigating-2.4-million-gallon-discrepancy-in-calhan-water-sales/article/1550718',
 'https://gazette.com/election/analysis-stapleton-coffman-don-t-expect-trump-s-help-before/article_fdac0ef6-abbf-11e8-a857-a7bb601fdaa2.html',
 'https://gazette.com/news/us-world/white-house-tries-to-clean-up-bidens-ukrainian-troops-training-comment/article_856d5fa1-7c6f-5c3e-919f-b49edcd53fd2.amp.html'

In [148]:
test = list(onsite)[0]
article = NewsPlease.from_url("http://gazette.com:80/article/1562984")
article.get

c:\Users\Nick\anaconda3\envs\archive\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gazette.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\Nick\anaconda3\envs\archive\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gazette.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [155]:
r = requests.get("http://gazette.com:80/article/1562984")
r.url

'https://gazette.com/news/vandals-unleashed-dogs-force-goat-herder-to-flee-el-paso-county-park/article_1320c72c-2da9-5975-8872-95b532a53911.html'

In [71]:
pub_dates = []
for k,v in tqdm(nopub.iterrows()):
    u = v.url_canonical
    if len(u)>0:
        r = requests.get(u)
        soup = BeautifulSoup(r.text)
        try:
            pub_dt = soup.find("time")['datetime']
        except (AttributeError, TypeError) as e:
            pub_dt = ""
        pub_dates.append(pub_dt)
    else:
        pub_dt = ""
        pub_dates.append(pub_dt)

3748it [47:54,  1.30it/s]


In [72]:
nopub.pub_date = pub_dates
nopub.drop(columns="index").to_sql("parsed_articles", con, if_exists="append")

# Mic

In [92]:
con = sqlite3.connect("./data/mic.db")
db = Database(con)
parsed = db.read_table("parsed_articles")
onsite = db.get_urls_from_table("onsite")
# check if pub date is none
nopub = parsed[(parsed.pub_date.isnull()) & (parsed.url.isin(onsite))]

In [81]:
pub_dates = []
for k,v in tqdm(nopub.iterrows()):
    u = v.url_canonical
    if len(u)>0:
        r = requests.get(u)
        soup = BeautifulSoup(r.text)
        try:
            pub_dt = soup.find("time")['datetime']
        except (AttributeError, TypeError) as e:
            pub_dt = ""
        pub_dates.append(pub_dt)
    else:
        pub_dt = ""
        pub_dates.append(pub_dt)

1375it [06:30,  3.52it/s]


In [83]:
nopub.pub_date = pub_dates
nopub.drop(columns="index").to_sql("parsed_articles", con, if_exists="append")

c:\Users\Nick\anaconda3\envs\archive\lib\site-packages\pandas\core\generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


# WSJ

In [95]:
con = sqlite3.connect("./data/wsj.db")
db = Database(con)
parsed = db.read_table("parsed_articles")
onsite = db.get_urls_from_table("onsite")
# check if pub date is none
nopub = parsed[(parsed.pub_date.isnull()) & (parsed.url.isin(onsite))]

In [98]:
onsite

{'https://www.wsj.com/articles/SB10001424052748704700204575643490042664712',
 'https://www.wsj.com/articles/three-injured-in-armed-attack-on-vienna-synagogue-11604348986',
 'http://www.wsj.com/articles/germanys-economic-resilience-should-reassure-investors-1448375218',
 'https://www.wsj.com/articles/SB10001424052748703514904575602992172574172',
 'https://www.wsj.com/articles/nvidia-simon-property-group-bny-mellon-what-to-watch-when-the-stock-market-opens-today-11605787409',
 'https://www.wsj.com/articles/SB10001424052748704141104575588462849667220',
 'https://www.wsj.com/articles/SB10001424052748703514904575602862946211030',
 'https://www.wsj.com/articles/educations-evolution-11605125339',
 'https://www.wsj.com/articles/SB10001424052748704312504575618660151287450',
 'http://www.wsj.com/articles/decorating-with-teal-the-color-people-love-to-hate-1446670265',
 'https://www.wsj.com/articles/SB10001424052748703514904575603462091864170',
 'https://www.wsj.com/articles/how-to-study-abroad-wh

In [ ]:
pub_dates = []
for k,v in tqdm(nopub.iterrows()):
    u = v.url_canonical
    if len(u)>0:
        r = requests.get(u)
        soup = BeautifulSoup(r.text)
        try:
            pub_dt = soup.find("time")['datetime']
        except (AttributeError, TypeError) as e:
            pub_dt = ""
        pub_dates.append(pub_dt)
    else:
        pub_dt = ""
        pub_dates.append(pub_dt)

In [ ]:
nopub.pub_date = pub_dates
nopub.drop(columns="index").to_sql("parsed_articles", con, if_exists="append")